In [6]:
from gensim.models.word2vec import Word2Vec
from gensim.models.phrases import Phrases, Phraser
import os, json, tqdm, re, itertools, nltk

In [7]:
def read_file(file_path):
    all_words_from_files = []
    with open (file_path) as file:
        json_content = json.load(file)
        item_count = 0
        for item in json_content:
            if item_count < 10:
                item_count += 1
                text_content = re.sub("<.*?>", "", item["textContent"])
                text_content = text_content.replace('-\n', '')
                text_content = text_content.replace("\n", '')
                
#                 print (text_content)
#             text_content = [word for word in text_content if len(word) > 0]
            all_words_from_files.append(text_content)
    return all_words_from_files
       
        
def read_all_judgments_from_2018(path):
    all_judgments = []
    for filename in tqdm.tqdm(os.listdir(path)):
        judgments = read_file(path + filename)
        all_judgments.append(judgments)
    return all_judgments

# We need a stream of sentences
def flatten_judgments(words):
    chain = list(itertools.chain(*words))
    result = []
    for judgment in chain:
        for word in judgment:
            result.append(word)
    return result
        

In [8]:
words = read_all_judgments_from_2018("../data_filtered_2/")

100%|██████████| 11/11 [00:00<00:00, 36.39it/s]


In [21]:
content = []
for judgment in words:
    for word in judgment:
        content.append(word)
content = " ".join(content)

'      UZASADNIENIE      Powód  (...) Sp. z o.o. Sp. k. z siedzibą w P. wniósł o zwolnienie spod egze'

In [27]:
from nltk.tokenize import TweetTokenizer, sent_tokenize
nltk.download('punkt')

tokenizer_words = TweetTokenizer()
tokens_sentences = [tokenizer_words.tokenize(t) for t in 
nltk.sent_tokenize(content)]


[nltk_data] Downloading package punkt to /home/grzegorz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [30]:
print(tokens_sentences[:10])

[['UZASADNIENIE', 'Powód', '(', '...', ')', 'Sp', '.'], ['z', 'o', '.', 'o', '.'], ['Sp', '.'], ['k', '.', 'z', 'siedzibą', 'w', 'P', '.', 'wniósł', 'o', 'zwolnienie', 'spod', 'egzekucji', 'administracyjnej', 'prawa', 'majątkowego', 'przysługującego', 'powodowi', 'w', 'postaci', 'wierzytelności', 'z', 'tytułu', 'nienależnie', 'pobranego', 'podatku', 'akcyzowego', 'za', 'miesiące', 'luty', ',', 'marzec', ',', 'kwiecień', ',', 'maj', ',', 'wrzesień', 'i', 'październik', '2011', 'r', '.', 'określonego', 'w', 'decyzji', 'Naczelnika', 'Urzędy', 'Celnego', 'w', 'Z', '.', 'z', 'dnia', '6', 'lutego', '2013', 'r', '.', 'nr', '(', '...', ')', '-', '(', '...', ')', '9110-414', '/', '12/13', '/', 'DN', 'oraz', 'Dyrektora', 'Izby', 'Celnej', 'w', 'R', '.', 'z', 'dnia', '8', 'kwietnia', '2014', 'r', '.', 'nr', '(', '...', ')', '-', '(', '...', ')', '-24/13', '/', '14', ',', 'która', 'został', 'stwierdzona', 'prawomocnym', 'wyrokiem', 'Wojewódzkiego', 'Sądu', 'Administracyjnego', 'w', 'G', '.', 'z', 

In [40]:
phrases = Phrases(tokens_sentences, threshold=0.5)

In [41]:
bigram = Phraser(phrases)

['Mam', 'dość', 'Wojewódzkiego', 'Sądu_Administracyjnego']
